In [237]:
import re
import numpy as np
import pandas as pd
import json
import shutil
import gzip
import os
import wget
import tarfile
import time
from time_converter import Time

- We will start with the users,receipts and the brands last
- We will look at the dataset and create structured data model

# User Data Cleaning

In [200]:
user_df = pd.read_csv('csv_files/users.csv', index_col = 0)

In [286]:
users_df = user_df.copy()

In [281]:
users_df.columns

Index(['_id', 'active', 'createdDate', 'lastLogin', 'role', 'signUpSource',
       'state'],
      dtype='object')

- We have a few Dtype objects in our dataset to observe for. While also checking the int64 and float64

In [203]:
users_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 495 entries, 0 to 494
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   _id           495 non-null    object
 1   active        495 non-null    bool  
 2   createdDate   495 non-null    object
 3   lastLogin     433 non-null    object
 4   role          495 non-null    object
 5   signUpSource  447 non-null    object
 6   state         439 non-null    object
dtypes: bool(1), object(6)
memory usage: 27.6+ KB


- Checking first 5 rows we see a few mor data that is in need of cleaning

In [204]:
users_df.head()

,_id,active,createdDate,lastLogin,role,signUpSource,state
0,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
1,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
2,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
3,{'$oid': '5ff1e1eacfcf6c399c274ae6'},True,{'$date': 1609687530554},{'$date': 1609687530597},consumer,Email,WI
4,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI


In [265]:
id_list = list(users_df[f'lastLogin'].to_dict().values())
date_parsed = id_list[0].replace('}', "").split(':')[1].replace(" ", "")
date_converted = Time(int(date_parsed[:10]), 'posix').to('dt')
format_date = "%m/%d/%Y"
format_time = "%H:%M:%S"
formated_date = date_converted.strftime(format_date)
formated_time = date_converted.strftime(format_time)

In [331]:
def cleanDateLog(id_data, column=None):
    id_list = list(id_data[f'{column}'].to_dict().values())
    times_list = []
    dates_list = []
    for i in range(len(id_list)):
        try: 
            
            date_parsed = id_list[i].replace('}', "").split(':')[1].replace(" ", "")
            date_converted = Time(int(date_parsed[:10]), 'posix').to('dt')
            format_date = "%m/%d/%Y"
            format_time = "%H:%M:%S"
            formated_date = date_converted.strftime(format_date)
            formated_time = date_converted.strftime(format_time)
            times_list.append(formated_time)
            dates_list.append(formated_date)
        except AttributeError:
            
            date_parsed = str(id_list[i])
            dates_list.append(date_parsed)
            
    return dates_list

In [295]:
def cleanIdCpg(id_data, column=None):
#     print(id_data[f'{column}'])
    # create the dictionary into a list to only get the values
    id_list = list(id_data[f'{column}'].to_dict().values())
    cleanedList = []
    for i in range(len(id_list)):
        parsed_name = id_list[i]
        # remove commas colons and split at quotations
        parsed_names = re.findall("'([^']*)'", id_list[i])
        # these were reoccuring strings that i saw
        cleanedList.append(parsed_names[-1])

    return cleanedList

In [332]:
users_df = user_df.copy()

users_df['_id'] = cleanIdCpg(id_data=users_df, column="_id")
users_df['createdDate'] = cleanDateLog(id_data=users_df, column="createdDate")
users_df["lastLogin"] = cleanDateLog(id_data=users_df, column="lastLogin")

# convert to pandas datetime 
users_df['createdDate'] = pd.to_datetime(users_df['createdDate'])
users_df["lastLogin"] = pd.to_datetime(users_df['lastLogin'])

In [348]:
users_df.head()

,_id,active,createdDate,lastLogin,role,signUpSource,state
0,5ff1e194b6a9d73a3a9f1052,True,2021-01-03,2021-01-03,consumer,Email,WI
1,5ff1e194b6a9d73a3a9f1052,True,2021-01-03,2021-01-03,consumer,Email,WI
2,5ff1e194b6a9d73a3a9f1052,True,2021-01-03,2021-01-03,consumer,Email,WI
3,5ff1e1eacfcf6c399c274ae6,True,2021-01-03,2021-01-03,consumer,Email,WI
4,5ff1e194b6a9d73a3a9f1052,True,2021-01-03,2021-01-03,consumer,Email,WI


# Receipt Data Cleaning

['createDate', 'dateScanned', 'finishedDate', 'modifyDate', 'pointsAwardedDate', 'purchaseDate']


In [428]:
# I observed and found thats needed to converted a quick script to loop through the Dates
def convertDates(df):
    new_df = df.copy()
    col_list = list(new_df.columns)
    dates_col = col_list[3:10]
    dates_col.remove(col_list[8])
#     print(dates_col)
    for dates in dates_col:
        df[f'{dates}'] = cleanDateLog(id_data=df, column=f"{dates}")
        
    return df

receipts_df = convertDates(receipt_df)

In [432]:
receipt_df = pd.read_csv('csv_files/receipts.csv', index_col = 0)
receipts_df = convertDates(receipt_df)

In [433]:
receipts_df['_id'] = cleanIdCpg(receipts_df, column = '_id')

In [515]:
 receipts_df['rewardsReceiptItemList'][0].split(",")

["[{'barcode': '4011'",
 " 'description': 'ITEM NOT FOUND'",
 " 'finalPrice': '26.00'",
 " 'itemPrice': '26.00'",
 " 'needsFetchReview': False",
 " 'partnerItemId': '1'",
 " 'preventTargetGapPoints': True",
 " 'quantityPurchased': 5",
 " 'userFlaggedBarcode': '4011'",
 " 'userFlaggedNewItem': True",
 " 'userFlaggedPrice': '26.00'",
 " 'userFlaggedQuantity': 5}]"]

In [540]:
this = receipts_df['rewardsReceiptItemList'][2].split(",")

In [545]:
this[0]

"[{'needsFetchReview': False"

In [544]:
re.findall("'([^']*)'", this[0])[1]

IndexError: list index out of range

In [522]:
######     receiptInfoList = []
for i in range(len( receipts_df['rewardsReceiptItemList'] ) ):
    this = receipts_df['rewardsReceiptItemList'][i].split(",")
    print(i)
    this_dictionary = {re.findall("'([^']*)'", this[0])[0]: re.findall("'([^']*)'", this[0])[1]}
    receiptInfoList.append(this_dictionary)
# print(pd.DataFrame([{re.findall("'([^']*)'", this)[0]: re.findall("'([^']*)'", this)[1]}]))

0
1
2


IndexError: list index out of range

# Brand Data Cleaning

In [444]:
def BrandIdCpg(id_data, column=None):
#     print(id_data[f'{column}'])
    # create the dictionary into a list to only get the values
    id_list = list(id_data[f'{column}'].to_dict().values())
    cleanedList = []
    for i in range(len(id_list)):
        parsed_name = id_list[i]
        # remove commas colons and split at quotations
        parsed_names = re.findall("'([^']*)'", id_list[i])
        # these were reoccuring strings that i saw
        remove_list = ['$ref', 'Cogs','$id', '$oid']
        # remove the extra strings and keep the actual id #
        res = [i for i in parsed_names if i not in remove_list]
        cleanedList.append(res[0])
        
    return cleanedList

In [446]:
brand_df = pd.read_csv('csv_files/brands.csv', index_col = 0)
brands_df = brand_df.copy()
brands_df['_id'] = BrandIdCpg(brands_df, column = '_id')
brands_df['cpg'] = BrandIdCpg(brands_df, column = 'cpg')
brands_df.head(10)

,_id,barcode,category,categoryCode,cpg,name,topBrand,brandCode
0,601ac115be37ce2ead437551,511111019862,Baking,BAKING,601ac114be37ce2ead437550,test brand @1612366101024,0.0,NaN
1,601c5460be37ce2ead43755f,511111519928,Beverages,BEVERAGES,5332f5fbe4b03c9a25efd0ba,Starbucks,0.0,STARBUCKS
2,601ac142be37ce2ead43755d,511111819905,Baking,BAKING,601ac142be37ce2ead437559,test brand @1612366146176,0.0,TEST BRANDCODE @1612366146176
3,601ac142be37ce2ead43755a,511111519874,Baking,BAKING,601ac142be37ce2ead437559,test brand @1612366146051,0.0,TEST BRANDCODE @1612366146051
4,601ac142be37ce2ead43755e,511111319917,Candy & Sweets,CANDY_AND_SWEETS,5332fa12e4b03c9a25efd1e7,test brand @1612366146827,0.0,TEST BRANDCODE @1612366146827
5,601ac142be37ce2ead43755b,511111719885,Baking,BAKING,601ac142be37ce2ead437559,test brand @1612366146091,0.0,TEST BRANDCODE @1612366146091
6,601ac142be37ce2ead43755c,511111219897,Baking,BAKING,601ac142be37ce2ead437559,test brand @1612366146133,0.0,TEST BRANDCODE @1612366146133
7,5cdad0f5166eb33eb7ce0faa,511111104810,Condiments & Sauces,NaN,559c2234e4b06aca36af13c6,J.L. Kraft,NaN,J.L. KRAFT
8,5ab15636e4b0be0a89bb0b07,511111504412,Canned Goods & Soups,NaN,5a734034e4b0d58f376be874,Campbell's Home Style,0.0,CAMPBELLS HOME STYLE
9,5c408e8bcd244a1fdb47aee7,511111504788,Baking,NaN,59ba6f1ce4b092b29c167346,test,NaN,TEST
